In [3]:
!pip install pandas


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
from collections import defaultdict

df = pd.read_csv('tubedata.csv', header=None)

station_dict = defaultdict(list)
line_dict = defaultdict(set)
zone_dict = defaultdict(set)

# get data row by row
for index, row in df.iterrows():
    start_station = row[0]
    end_station = row[1]
    line = row[2]
    act_cost = int(row[3])
    zone1 = row[4]
    zone2 = row[5]

    # station dictionary of child station tuples
    # (child_name, cost from parent to the child)
    # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
    
    # Add connection from start_station to end_station
    station_dict[start_station].append((end_station, act_cost))
    
    # filter abnormal data in tubedata.cvs
    if(line == 'a'):
        line = 7
    elif(line == 'b'):
        line = 8
    elif(line == 'c'):
        line = 9
    elif(line == 'd'):
        line = 10
    line_dict[start_station].add(line)

    # Add connection from end_station to start_station
    station_dict[end_station].append((start_station, act_cost))

    # Add main zone for start_station
    zone_dict[start_station] = (zone1)

    # Add secondary zone if not "0" for start_station
    if zone2 != 0:
        # If zone2 is not "0", it’s the main zone for the ending station
        zone_dict[end_station] = (zone2)
    else:
        # Otherwise, the main zone for the ending station is the same as for the starting station
        zone_dict[end_station] = (zone1)



<br>








<br>

In [5]:
def show_result(path, num_expanded_nodes, start_station, goal_station):
    if path:
        print(f"Path from {start_station} to {goal_station}:")
        line_changes = 0
        current_lines = set()
        lines_taken = []

        for i in range(1, len(path)):
            current_station, current_cost = path[i - 1]
            next_station, next_cost = path[i]

            if not current_lines:
                current_lines = line_dict[current_station]
                lines_taken.append('/'.join(current_lines))

            neighbor_lines = line_dict[next_station]
            common_lines = current_lines.intersection(neighbor_lines)

            if not common_lines:
                line_changes += 1
                current_lines = line_dict[next_station]
                lines_taken.append('/'.join(current_lines))

        cumulative_cost = sum(cost for _, cost in path)
        print(f"Depth: {len(path)}")
        print(f"Cumulative Time Cost: {cumulative_cost}")
        print(f"Number of Nodes Expanded: {num_expanded_nodes}")
        print(f"Number of Line Changes: {line_changes}")
        print(f"Lines Taken: {' -> '.join(lines_taken)}")
        print(f"Zones Traveled: {zone_dict[start_station]} to {zone_dict[goal_station]}")
    else:
        print(f"No path found from {start_station} to {goal_station}")


In [6]:
from collections import deque

def bfs(station_dict, start, goal):
    frontier = deque([(start, [])])  # Using a deque for efficient pop from left (popleft)
    explored = set()
    num_expanded_nodes = 0

    while frontier:
        current_station, current_path = frontier.popleft()
        num_expanded_nodes += 1

        if current_station == goal:
            return current_path, num_expanded_nodes

        if current_station not in explored:
            explored.add(current_station)
            for neighbor, cost in station_dict[current_station]:
                frontier.append((neighbor, current_path + [(current_station, cost)]))

    return None, num_expanded_nodes

# Example usage:
start_station = 'Euston'
goal_station = 'Victoria'

start_station = 'Canary Wharf'
goal_station = 'Stratford'

start_station = 'Ealing Broadway'
goal_station = 'South Kensington'

path, num_expanded_nodes = bfs(station_dict, start_station, goal_station)

show_result(path, num_expanded_nodes, start_station, goal_station)

Path from Ealing Broadway to South Kensington:
Depth: 8
Cumulative Time Cost: 20
Number of Nodes Expanded: 115
Number of Line Changes: 0
Lines Taken: District/Central
Zones Traveled: 3 to 1


In [7]:
def dfs(station_dict, start, goal, reverse):
    stack = [(start, [])]  # Using a list as a stack
    explored = set()
    num_expanded_nodes = 0

    while stack:
        if (reverse and num_expanded_nodes == 1):
            #if reverse = true and reverse the first explore branch
            stack = stack[::-1]  # Reverse the stack

        current_station, current_path = stack.pop()
        num_expanded_nodes += 1

        if current_station == goal:
            return current_path, num_expanded_nodes

        if current_station not in explored:
            explored.add(current_station)
            for neighbor, cost in station_dict[current_station]:
                stack.append((neighbor, current_path + [(current_station, cost)]))

    return None, num_expanded_nodes

# Example usage:
start_station = 'Euston'
goal_station = 'Victoria'


# start_station = 'Embankment'
# goal_station = 'Mile End'
start_station = 'Ealing Broadway'
goal_station = 'South Kensington'

# start_station = 'Leicester Square'
# goal_station = 'Stepney Green'


reverse = True
path, num_expanded_nodes = dfs(station_dict, start_station, goal_station, reverse)

show_result(path, num_expanded_nodes, start_station, goal_station)

Path from Ealing Broadway to South Kensington:
Depth: 33
Cumulative Time Cost: 87
Number of Nodes Expanded: 93
Number of Line Changes: 3
Lines Taken: District/Central -> Hammersmith & City -> Metropolitan/Jubilee -> Piccadilly
Zones Traveled: 3 to 1


In [8]:
import heapq

def ucs(station_dict, start, goal, line_dict, cost_change_of_line):
    priority_queue = [(0, start, [])]  # Using a heap queue
    explored = set()
    num_expanded_nodes = 0

    while priority_queue:
        #heapq.heappop will output the lowest cost neighbour as the next expanding node
        current_cost, current_station, current_path = heapq.heappop(priority_queue)
        num_expanded_nodes += 1
        if(current_station == start):
            current_lines = line_dict[start]

        if current_station == goal:
            return current_path, num_expanded_nodes

        if current_station not in explored:
            explored.add(current_station)
            for neighbor, cost in station_dict[current_station]:
                neighbor_lines = line_dict[neighbor]
                common_lines = current_lines.intersection(neighbor_lines)
                new_cost = current_cost + cost
                if not common_lines:
                    current_lines = line_dict[neighbor]
                    # add neighbout with the updated cost in to the queue
                    heapq.heappush(priority_queue, (new_cost + cost_change_of_line, neighbor, current_path + [(current_station, cost + cost_change_of_line)]))
                else:
                    heapq.heappush(priority_queue, (new_cost, neighbor, current_path + [(current_station, cost)]))
    return None, num_expanded_nodes


start_station = 'Euston'
goal_station = 'Victoria'

start_station = 'Leicester Square'
goal_station = 'Ealing Broadway'


# the cost in minute when change of line 0 to disable extended
cost_change_of_line = 4
path, num_expanded_nodes = ucs(station_dict, start_station, goal_station, line_dict, cost_change_of_line)

show_result(path, num_expanded_nodes, start_station, goal_station)

Path from Leicester Square to Ealing Broadway:
Depth: 13
Cumulative Time Cost: 29
Number of Nodes Expanded: 439
Number of Line Changes: 0
Lines Taken: Northern/Piccadilly
Zones Traveled: 1 to 3


In [9]:
import heapq

def bfs_heuristic(station_dict, start, goal, line_dict, zone_dict):
    priority_queue = [(0, start, [])]  # Using a heap queue
    explored = set()
    num_expanded_nodes = 0

    while priority_queue:
        current_cost, current_station, current_path = heapq.heappop(priority_queue)
        num_expanded_nodes += 1
        if(current_station == start):
            current_lines = line_dict[start]

        if current_station == goal:
            return current_path, num_expanded_nodes

        if current_station not in explored:
            explored.add(current_station)
            current_lines = line_dict[current_station]
            
            for neighbor, cost in station_dict[current_station]:
                # Calculate the heuristic value based on zone changes to the goal
                heuristic_value = (zone_dict[goal] - zone_dict[neighbor])**2
                if heuristic_value >= 1:
                    heuristic_value += 15
                neighbor_lines = line_dict[neighbor]
                common_lines = current_lines.intersection(neighbor_lines)
                if not common_lines:
                    cost += 3
                    current_lines = line_dict[neighbor]
                # Calculate the priority using the heuristic value
                new_cost = current_cost + cost
                priority = heuristic_value + new_cost
                heapq.heappush(priority_queue, (priority, neighbor, current_path + [(current_station, cost)]))

    return None, num_expanded_nodes

# Example usage:
start_station = 'Euston'
goal_station = 'Victoria'

start_station = 'Leicester Square'
goal_station = 'Stepney Green'

# start_station = 'Embankment'
# goal_station = 'Mile End'

start_station = 'Leicester Square'
goal_station = 'Ealing Broadway'

# Call the BFS algorithm with the heuristic
path, num_expanded_nodes = bfs_heuristic(station_dict, start_station, goal_station, line_dict, zone_dict)

show_result(path, num_expanded_nodes, start_station, goal_station)

Path from Leicester Square to Ealing Broadway:
Depth: 13
Cumulative Time Cost: 28
Number of Nodes Expanded: 531
Number of Line Changes: 1
Lines Taken: Northern/Piccadilly -> District/Victoria/Circle
Zones Traveled: 1 to 3


In [16]:
import math
import hashlib
import string

def get_password(student_username, l=10):
    # Possible characters include upper-case English letters, numbers between 0 and 9 (inclusive), 
    # and the underscore symbol
    options = string.digits + string.ascii_uppercase  + "_"

    h = hashlib.sha256(("ECS759P-AI"+student_username).encode("utf-8"))
    d = h.digest()
    s = ""
    for n in d:
      s += options[n%len(options)]

    return s[0:l]

# TO DO: replace *** with your EECS username and uncomment the code
student_password = get_password('230689365')
print(student_password)

WNZJITMQ3C


In [17]:
# Distance function
def distance_function(string_one, string_two):
    score = 0
    for i, j in zip(string_one, string_two):
        # Square of the absolute difference between two Unicode codes
        score += math.sqrt(abs(ord(i) - ord(j)))
    return score


# Upper bound of the distance value
MAX_VALUE = distance_function('0000000000', '__________')

# Compute normalised fitness for a list of candidate passwords 
def get_normalised_fitness(list_of_phrases, student_password):
    ordered_dict = dict()
    phrase_to_find = student_password
    for phrase in list_of_phrases:
        # Return 1 when a candidate matches the true password (string distance between them is zero)
        ordered_dict[phrase] = 1 - distance_function(phrase, phrase_to_find) / MAX_VALUE
    return ordered_dict

# Example of how to get fitness values for a list of candidates
get_normalised_fitness(['2Q4HHHHOTJ', '2HHZQYUOTJ'], student_password)

{'2Q4HHHHOTJ': 0.5347178369523395, '2HHZQYUOTJ': 0.4971692024106349}

In [33]:
import numpy as np
import random

options = string.digits + string.ascii_uppercase + "_"

def create_init_population(length):
    '''
    Method to create initial population
    '''
    random_password = ''.join(random.choice(options) for _ in range(length))
    return random_password

    
def mutation(password, pMuta = 0.5):
    """
    Mutate provided individuals with pMuta probability
    """
    mutated_password = ""
    for char in password:
        if np.random.rand() < pMuta:
            mutated_password += random.choice(options)
        else:
            mutated_password += char
      # TO DO
    return mutated_password


def crossing_over(password1, password2):
    # Randomly choose a crossover point
    crossover_point = random.randint(0, len(password1) - 1)
    # Perform crossover
    new_password = password1[:crossover_point] + password2[crossover_point:]
    return new_password

def crack_password(actual_password, population_size, mutation_rate, crossover_rate, max_generations):
    # Generate initial population
    population = [create_init_population(len(actual_password)) for _ in range(population_size)]
    no_of_reproduction = 0

    for generation in range(max_generations):
        # Evaluate fitness for each candidate password
        fitness_values = get_normalised_fitness(population, actual_password)
        sorted_population = sorted(population, key=lambda x: fitness_values[x], reverse=True)

        # Check if the password is found
        if actual_password in sorted_population:
            print("Number of reproduction:", no_of_reproduction)
            print("Password found in generation:", generation)
            return no_of_reproduction
        if(generation == max_generations-1):
            print("Fail to crack password within max generation")
            return no_of_reproduction
        # Select best-performing individuals (parents)
        mating_pool = sorted_population[:population_size // 2]

        # Create new generation through crossover and mutation
        new_population = []
        while len(new_population) < population_size:
            parent1, parent2 = random.choices(mating_pool, k=2)
            
            # Apply crossover
            if np.random.rand() <= crossover_rate:
                """
                Applies crossing over on the selected individuals with a probability crossover_rate
                """
                child= crossing_over(parent1, parent2)
                no_of_reproduction += 1
            else:
                child = parent1
            
            # Apply mutation
            child = mutation(child, mutation_rate)
            
            new_population.append(child)
        
        population = new_population

# Get the actual password
student_username = '230689365'
actual_password = get_password(student_username)
print(f"Actual Password: {actual_password}")

# Crack the password using genetic algorithm parameters
population_size = 100
mutation_rate = 0.1
crossover_rate = 0.5
max_generations = 1000
no = crack_password(actual_password, population_size, mutation_rate, crossover_rate, max_generations)

Actual Password: WNZJITMQ3C
Number of reproduction: 10639
Password found in generation: 215


In [ ]:
# Parameters for experiments
runs = 10
reproduction_counts = []
population_size = 100
mutation_rate = 0.1
crossover_rate = 0.5
max_generations = 1000

# Run the cracking process multiple times
for _ in range(runs):
    no_of_reproduction = 0  # Reset reproduction count for each run
    no = crack_password(actual_password, population_size, mutation_rate, crossover_rate, max_generations)
    reproduction_counts.append(no)  # Store the reproduction count for this run

# Calculate average and standard deviation
average_reproductions = np.mean(reproduction_counts)
std_dev_reproductions = np.std(reproduction_counts)

# Print the results
print(f"Average number of reproductions: {average_reproductions}")
print(f"Standard deviation of reproductions: {std_dev_reproductions}")

Number of reproduction: 5787
Password found in generation: 114
Number of reproduction: 4871
Password found in generation: 99
Number of reproduction: 4459
Password found in generation: 90
Number of reproduction: 4699
Password found in generation: 94
Number of reproduction: 5619
Password found in generation: 113
Number of reproduction: 8772
Password found in generation: 175
Number of reproduction: 5218
Password found in generation: 104
Number of reproduction: 8531
Password found in generation: 170
Number of reproduction: 3515
Password found in generation: 70
Number of reproduction: 4679
Password found in generation: 94
Average number of reproductions: 5615.0
Standard deviation of reproductions: 1633.1545548416414
